# Relatório sobre o Código de Raspagem de Notícias do G1

## Objetivo
O código tem como objetivo coletar notícias do site G1 relacionadas a eventos climáticos (chuvas, ciclones, tempestades, etc.) usando Selenium e BeautifulSoup. Ele extrai os seguintes campos de cada notícia:
- **Produto**
- **Link**
- **Título**
- **Data**
- **Conteúdo**

## Funcionalidade

### 1. **Bibliotecas Importadas**
- **BeautifulSoup**: Para parsear o HTML e extrair dados.
- **Selenium**: Para interagir com a página dinâmica.
- **pandas**: Para estruturar e salvar os dados.
- **tqdm**: Para exibir barra de progresso.
- **time**: Para medir o tempo de execução.
- **urllib.parse**: Para tratar URLs.

### 2. **Função Principal: `scrape_infinite_scroll`**
- **Entrada**: URL da pesquisa e número de itens a serem coletados.
- **Processo**:
  - Navega pelas páginas do G1 utilizando Selenium.
  - Coleta os blocos de notícias com BeautifulSoup.
  - Para cada bloco, extrai:
    - **Produto**: Categoria da notícia.
    - **Link**: URL da página completa.
    - **Título**: Título da notícia.
    - **Data**: Data da publicação.
    - **Conteúdo**: Descrição resumida.
  - Repete o processo até coletar o número especificado de notícias.

### 3. **Configuração do Selenium**
- Configurações para rodar em modo "headless" (sem interface gráfica).
- Configuração de "user-agent" para simular navegação real.
- Navegação nas páginas do G1 com a URL dinâmica (para carregar mais notícias).

### 4. **Execução do Processo**
- O usuário define o número de notícias a coletar e o intervalo de anos.
- O código gera a URL de pesquisa para cada ano e coleta as notícias.
- Os dados são armazenados em um DataFrame `df` e exportados para um arquivo Excel.

### 5. **Controle de Tempo**
- O tempo de execução da raspagem é calculado e exibido.

## Campos Coletados
- **Produto**: Categoria da notícia (ex: "Previsão do Tempo").
- **Link**: URL da notícia.
- **Título**: Título da notícia.
- **Data**: Data de publicação.
- **Conteúdo**: Resumo da notícia.

## Fluxo de Execução
1. O script solicita ao usuário o número de notícias e o intervalo de anos.
2. Realiza buscas no G1 com palavras-chave (ex: "Chuva, Ciclone").
3. Navega pelas páginas de resultados e coleta os dados.
4. Armazena e exporta os dados para um arquivo Excel.

## Pontos de Melhoria
- **Tratamento de Exceções**: Melhorar a captura de erros específicos para tornar a raspagem mais robusta.
- **Otimização**: Evitar carregamento desnecessário de páginas.
- **Validação de Dados**: Verificar se os campos coletados estão completos antes de salvar.
- **Uso de Proxies/Delay**: Para evitar bloqueios no site G1.

## Conclusão
O código realiza a raspagem de notícias de forma eficiente, coletando informações sobre eventos climáticos do site G1. Ele organiza os dados em um DataFrame e exporta para um arquivo Excel, facilitando a análise posterior.

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from tqdm import tqdm
from urllib.parse import unquote

def scrape_infinite_scroll(url, num_items):
    start_time = time.time()  

    driver.get(url)

    scraped_data = []
    noticia = {""}

    page_counter = 1

    with tqdm(total=num_items, desc="Raspando dados") as pbar:
        
        while len(scraped_data) < num_items:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            product_blocks = soup.find_all('li', {'class': 'widget widget--card widget--info'})
            
            for block in product_blocks:    
                if not block.find('a').get('href'):
                    continue
                try:
                    if len(scraped_data) >= num_items:
                        pbar.update(1)
                        break
                    noticia = {"Produto": block.find('div', class_='widget--info__header').text.strip(),
                               "Link": unquote("https" + block.find('a').get('href').split("https")[1]).split("&syn")[0],
                               "Título": block.find('div', 'widget--info__title').text.strip(),
                               "Data": block.find('div', 'widget--info__meta').text.strip(),
                               "Conteúdo": block.find('p', class_='widget--info__description').text.strip()
                              }
                    
                    scraped_data.append(noticia)
                except:
                    pass
                pbar.update(1)


            page_counter += 1
            driver.get(url + f"&page={page_counter}")


    end_time = time.time()  # Captura o tempo de término
    total_time = end_time - start_time  # Calcula o tempo total de execução
    
    print(f"Tempo total de execução: {total_time:.2f} segundos")

    return scraped_data

if __name__ == '__main__':
    
    num_items = int(input('Digite o número de notícias que deseja coletar por ano: '))
    
    palavra_chave = "Chuva, Temporal, Tempestade, Ciclone, Previsão, Frente fria"
    
    ano_inicial = int(input("Ano inicial da pesquisa: "))
    ano_final = int(input("Ano final da pesquisa: "))

    arquivo = input("Arquivo para continuar")

    print("Iniciando selenium...")
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-web-security')
    options.add_argument('--disable-site-isolation-trials')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument('--disable-notifications')

    print("Configurando user agent...")
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')
    
    print("Iniciando webdriver...")
    driver = webdriver.Chrome(options=options)

        
    print("Inicializando dataFrame...")
    if arquivo != "":
        df = pd.read_excel(arquivo)
    else:
        df = pd.DataFrame(columns=["Produto", "Link", "Título","Data","Conteúdo"])
    
    scraped_data = []
    for ano in range(ano_inicial, ano_final+1):
        url = f"https://g1.globo.com/busca/?q={palavra_chave}&order=recent&from={ano}-01-01T00%3A00%3A00-0200&to={ano}-12-30T23%3A59%3A59-0200"  # URL com palavra-chave
        print(f"\nBuscando {ano}")
        scraped_data = scrape_infinite_scroll(url, num_items)

        dfTemp = pd.DataFrame(list(map(lambda x: x.values(), scraped_data)) ,columns=scraped_data[0].keys())
        df = pd.concat([df, dfTemp])
        
        df.to_excel('./planilhas/Noticias_cruas_duplicatas.xlsx', index=False)
    
    driver.quit()

    # Convertendo os dados para um DataFrame do pandas
    
    # Salvando o DataFrame em um arquivo Excel
    print("Raspagem de dados finalizada")


Esse código a baixo é uma variação do primeiro para realizar testes no ano de 2020 com cada palavra chave sendo buscada individualmente

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from tqdm import tqdm
from urllib.parse import unquote

def scrape_infinite_scroll(url, num_items):
    start_time = time.time()  

    driver.get(url)

    scraped_data = []
    noticia = {""}

    page_counter = 1

    with tqdm(total=num_items, desc="Raspando dados") as pbar:
        
        while len(scraped_data) < num_items:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            product_blocks = soup.find_all('li', {'class': 'widget widget--card widget--info'})
            
            for block in product_blocks:    
                if not block.find('a').get('href'):
                    continue
                try:
                    if len(scraped_data) >= num_items:
                        pbar.update(1)
                        break
                    noticia = {"Produto": block.find('div', class_='widget--info__header').text.strip(),
                               "Link": unquote("https" + block.find('a').get('href').split("https")[1]).split("&syn")[0],
                               "Título": block.find('div', 'widget--info__title').text.strip(),
                               "Data": block.find('div', 'widget--info__meta').text.strip(),
                               "Conteúdo": block.find('p', class_='widget--info__description').text.strip()
                              }
                    
                    scraped_data.append(noticia)
                except:
                    pass
                pbar.update(1)


            # Avança para a próxima página
            page_counter += 1
            driver.get(url + f"&page={page_counter}")


    end_time = time.time()  # Captura o tempo de término
    total_time = end_time - start_time  # Calcula o tempo total de execução
    
    print(f"Tempo total de execução: {total_time:.2f} segundos")

    return scraped_data

if __name__ == '__main__':
    
    num_items = int(input('Digite o número de notícias que deseja coletar por ano: '))
    
    palavras_chave = ["Chuva", "Temporal", "Tempestade", "Ciclone", "Previsão", "Frente fria"]
    
    ano_inicial = int(input("Ano inicial da pesquisa: "))
    ano_final = int(input("Ano final da pesquisa: "))

    arquivo = input("Arquivo para continuar")

    print("Iniciando selenium...")
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-web-security')
    options.add_argument('--disable-site-isolation-trials')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument('--disable-notifications')

    print("Configurando user agent...")
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')
    
    print("Iniciando webdriver...")
    driver = webdriver.Chrome(options=options)

        
    print("Inicializando dataFrame...")
    if arquivo != "":
        df = pd.read_excel(arquivo)
    else:
        df = pd.DataFrame(columns=["Produto", "Link", "Título","Data","Conteúdo"])
    
    scraped_data = []
    for chave in palavras_chave:
        url = f"https://g1.globo.com/busca/?q={chave}&order=recent&from={2020}-01-01T00%3A00%3A00-0200&to={2020}-12-30T23%3A59%3A59-0200"  # URL com palavra-chave
        print(f"\nBuscando {chave}")
        scraped_data = scrape_infinite_scroll(url, num_items)

        dfTemp = pd.DataFrame(list(map(lambda x: x.values(), scraped_data)) ,columns=scraped_data[0].keys())
        df = pd.concat([df, dfTemp])
        
        df.to_excel('./planilhas/Noticias_cruas_duplicatas.xlsx', index=False)
    
    driver.quit()

    # Convertendo os dados para um DataFrame do pandas
    
    # Salvando o DataFrame em um arquivo Excel
    print("Raspagem de dados finalizada")


## Este código remove as notícias duplicadas de uma planilha, considerando o título como critério para identificação de duplicidade. Ele carrega os dados de um arquivo Excel, calcula o número de títulos únicos, remove as duplicatas e salva o resultado em um novo arquivo Excel. A quantidade final de registros após a remoção das duplicatas é exibida.

In [ ]:
import pandas as pd

df = pd.read_excel("./planilhas/Noticias_cruas_duplicatas.xlsx")

print(len(list(set(df["Título"]))))

print(len(df))

noticias_filtradas = df.drop_duplicates()

print(f'Ficaram {len(noticias_filtradas)} resgistros de notícias após a retirada das notícia duplicicadas.')

noticias_filtradas.to_excel("./planilhas/Noticias_sem_duplicadas.xlsx", index=False)
display(noticias_filtradas)

### Mini Relatório: Análise do Código

**Objetivo:** Exibir a quantidade de notícias encontradas por ano (2006–2024) a partir de datas em um arquivo Excel.  

**Funcionamento:**
1. **Importação:** Usa a biblioteca `pandas` para manipular dados da planilha.
2. **Leitura:** Lê a planilha `Noticias_sem_duplicadas.xlsx` e converte a coluna "Data" em uma lista.
3. **Criação do Dicionário:** Inicializa um dicionário com anos (2006–2024) como chaves e contagens iniciadas em 0.
4. **Extração de Anos:** Usa `lambda` para capturar os 4 caracteres do ano de cada data no formato "DD/MM/AAAA".
5. **Contagem:** Incrementa a contagem de notícias no ano correspondente, ignorando datas malformadas.
6. **Saída:** Exibe o dicionário com a contagem de notícias por ano. 

**Exemplo de saída:**  
```python
{'2024': 5, '2023': 12, ..., '2006': 1}
```

In [ ]:
import pandas as pd

temp = pd.read_excel("./planilhas/Noticias_sem_duplicadas.xlsx")
temp = list(temp["Data"])
a = {str(x):0 for x in list(range(2006,2025))[::-1]}
for x in list(map(lambda x: x[6:10], temp)):
    try:
        a[x] += 1
    except:
        pass
print(a)


### Mini Relatório: Análise do Código

**Objetivo:** Identificar e associar notícias a cidades e/ou regiões de Santa Catarina (SC) através de análise textual no título e conteúdo das notícias.

---

**Funcionamento:**  
1. **Importação e Leitura:**  
   - Lê as listas de cidades (arquivo `cidade_sc1.xlsx`) e notícias (`Noticias_sem_duplicadas.xlsx`).

2. **Busca Textual:**  
   - Verifica, para cada notícia, se cidades ou regiões aparecem no título ou conteúdo.  
   - Filtra entradas com dados inválidos.

3. **Atribuição de Região:**  
   - Associa a cidade encontrada a uma região correspondente se nenhuma região foi explicitamente mencionada.

4. **Atualização:**  
   - Adiciona colunas "Cidade" e "Região" ao DataFrame de notícias.

5. **Exportação:**  
   - Salva o DataFrame atualizado no arquivo `Notícias_cidade_regiao_definida.xlsx`.

**Saída:**  
Notícias classificadas por cidade e região com dados consistentes.

In [ ]:
import pandas as pd
cidades_sc1 = pd.read_excel('./planilhas/cidade_sc1.xlsx')
noticias_filtradas = pd.read_excel("./planilhas/Noticias_sem_duplicadas.xlsx")

# Loop sobre as notícias
regioes = ['SUL', 'VALE DO ITAJAÍ', 'NORTE', 'GRANDE FLORIANÓPOLIS', 'SERRANA', 'OESTE']
for index, noticia in noticias_filtradas.iterrows():
    cidade_encontrada = []  # Valor padrão caso nenhuma cidade seja encontrada na notícia
    regiao_encontrada = []
    # Loop sobre as cidades de Santa Catarina
    if type(noticia["Título"]) != str or type(noticia["Conteúdo"]) != str:
            continue
        
    for cidade in cidades_sc1['MUNICIPIO']:
        if cidade.strip().upper() in noticia['Título'].upper() or cidade.strip().upper() in noticia["Conteúdo"].upper():  
            cidade_encontrada.append(cidade.strip()) 

    for regiao in regioes:
        if regiao.strip().upper() in noticia["Título"].upper() or regiao.strip().upper() in noticia["Conteúdo"].upper():
            regiao_encontrada.append(regiao)


    if cidade_encontrada and not regiao_encontrada:
        for cidade in cidade_encontrada:
            idx = list(map(lambda x: x[0],cidades_sc1.values)).index(cidade) 
            regiao_encontrada.append(cidades_sc1.values[idx][1])
            
    noticias_filtradas.at[index, 'Cidade'] = str(list(set(cidade_encontrada)))
    noticias_filtradas.at[index, "Região"] = str(list(set(regiao_encontrada)))

noticias_filtradas.to_excel('./planilhas/Notícias_cidade_regiao_definida.xlsx', index=False)

len(noticias_filtradas)

#display(noticias_filtradas)


### Mini Relatório: Análise do Código

**Objetivo:**  
Filtrar as notícias relacionadas a Santa Catarina (SC) utilizando palavras-chave e nomes de cidades do estado. Esse filtro complementa o anterior e avalia a eficácia de diferentes abordagens.

---

**Funcionamento:**  

1. **Importação e Preparação:**  
   - Lê os arquivos:
     - **Notícias:** `Noticias_sem_duplicadas.xlsx`.  
     - **Cidades SC:** `cidade_sc1.xlsx`.  
   - Inicializa a lista `a` com valores `False` para marcar notícias que serão mantidas.

2. **Filtragem no Título:**  
   - Verifica se os títulos das notícias contêm:
     - Palavras-chave: `"santa catarina"`, `"santa-catarina"`, ou `" sc "`.  
     - Nomes de cidades da lista.  
   - Atualiza a lista `a` para `True` caso qualquer condição seja atendida.

3. **Filtragem no Conteúdo:**  
   - Similar ao título, mas aplicado ao texto do conteúdo, ignorando valores não textuais.

4. **Aplicação do Filtro:**  
   - Retém no DataFrame apenas as notícias marcadas como relevantes (`True`).

5. **Exportação:**  
   - Salva o DataFrame filtrado no arquivo `Filtragem_palavra_individual_cidades.xlsx`.

---

**Saída:**  
Um novo arquivo contendo apenas as notícias filtradas relacionadas a SC com base em palavras-chave e nomes de cidades.

In [ ]:
import pandas as pd
noticias_filtradas = pd.read_excel("./planilhas/Noticias_sem_duplicadas.xlsx")
cidades_sc1 = pd.read_excel('./planilhas/cidade_sc1.xlsx')
print(len(noticias_filtradas))


a = [False] * len(noticias_filtradas)

for i, noticia in enumerate(noticias_filtradas["Título"]):
    if "santa catarina" in noticia.lower() or "santa-catarina" in noticia.lower() or " sc " in noticia.lower():
        a[i] = True
    for cidade in cidades_sc1["MUNICIPIO"]:
        if cidade.strip().lower() in noticia.lower():
            a[i] = True
            break
            
for i, noticia in enumerate(noticias_filtradas["Conteúdo"]):
    if type(noticia) != str:
        continue
    if "santa catarina" in noticia.lower() or "santa-catarina" in noticia.lower() or " sc " in noticia.lower():
        a[i] = True
    for cidade in cidades_sc1["MUNICIPIO"]:
        if cidade.strip().lower() in noticia.lower():
            a[i] = True
            break

noticias_filtradas = noticias_filtradas[a]  



# noticias_filtradas.to_excel('Notícias_cidade_regiao_definida.xlsx', index=False)
noticias_filtradas.to_excel('./planilhas/Filtragem_palavra_individual_cidades.xlsx', index=False)

len(noticias_filtradas)

#display(noticias_filtradas)


### Mini Relatório: Análise do Código

**Objetivo:**  
Filtrar as notícias que mencionam Santa Catarina e excluir as que fazem referência a outros estados brasileiros, usando palavras-chave, links e informações regionais.

---

**Funcionamento:**  

1. **Importação e Leitura de Dados:**  
   - Lê o arquivo `Notícias_cidade_regiao_definida.xlsx`, contendo notícias pré-filtradas, e seleciona aquelas com links válidos.  

2. **Primeiro Filtro:**  
   - Retém notícias que:
     - Contêm "santa-catarina", "/sc/", ou links associados à SC.
     - Possuem cidades definidas ou pertencem ao produto "G1".

3. **Checagem de Estados:**  
   - Analisa o texto das notícias e links buscando nomes ou siglas de estados diferentes de SC.
   - Caso outro estado seja encontrado, a notícia é marcada como irrelevante.

4. **Cálculo e Exportação:**  
   - Calcula o número e percentual de notícias relevantes filtradas.
   - Salva o resultado no arquivo `Filtradas link SC.xlsx`.

---

**Saída:**  
- **Total de notícias originais:** Número de entradas antes do filtro.  
- **Notícias relevantes:** Quantidade e percentual de notícias relacionadas a SC após o filtro.  
- **Exportação:** Arquivo Excel com as notícias relevantes.

In [ ]:
import pandas as pd
from urllib.parse import unquote

noticias_filtradas = pd.read_excel("./planilhas/Notícias_cidade_regiao_definida.xlsx")

filtered_df = noticias_filtradas[list(map(lambda x: "https" in x , noticias_filtradas["Link"]))]


RegioesBrasil = ['DF', 'Distrito Federal', 'GO', 'Goiás', 'MT', 'Mato Grosso', 'MS', 'Mato Grosso do Sul', 'Região Nordeste', 'AL', 'Alagoas', 'BA', 'Bahia', 'CE', 'Ceará', 'MA', 'Maranhão', 'PB', 'Paraíba', 'PE', 'Pernambuco', 'PI', 'Piauí', 'RN', 'Rio Grande do Norte', 'SE', 'Sergipe', 'Região Norte', 'AC', 'Acre', 'AP', 'Amapá', 'AM', 'Amazonas', 'PA', 'Pará', 'RO', 'Rondônia', 'RR', 'Roraima', 'TO', 'Tocantis', 'Região Sudeste', 'ES', 'Espírito Santo', 'MG', 'Minas Gerais', 'RJ', 'Rio de Janeiro', 'SP', 'São Paulo', 'PR', 'Paraná', 'RS', 'Rio Grande do Sul']

filtered_df = filtered_df[list(map(lambda x, y, z: z == "G1" or y != "[]" or "santa-catarina" in x.split("https")[1] or "/sc/" in x, filtered_df["Link"], filtered_df["Cidade"], filtered_df["Produto"]))]
print(len(filtered_df))
outraRegiao = [True]*len(filtered_df)

for noticia in range(len(filtered_df)):
    text = str(filtered_df.values[noticia][2]) + str(filtered_df.values[noticia][4])
    link = filtered_df["Link"][noticia]

    for x in RegioesBrasil:
        if x in text:
            outraRegiao[noticia] = False
            break
        if len(x) == 2:
            if "F" + x.lower() + "%" in link:
                outraRegiao[noticia] = False
                print(link)
        elif x.lower() in link or x.lower().replace(" ", "-") in link:
                outraRegiao[noticia] = False

# if filtered_df.values[noticia][]
filtered_df = filtered_df[outraRegiao]

total_noticias = len(noticias_filtradas)

num_noticias_filtradas = len(filtered_df)

percentual_filtradas = (num_noticias_filtradas / total_noticias) * 100

noticias_filtradas = filtered_df
noticias_filtradas.to_excel('./planilhas/Filtradas link SC.xlsx', index=False)

print(f"Total de notícias: {total_noticias}")
print(f"Número de notícias 'G1' ou 'G1 SC': {num_noticias_filtradas}")
print(f"Percentual correspondente: {percentual_filtradas:.2f}%")

### Mini Relatório: Análise do Código

**Objetivo:**  
Realizar a última filtragem de notícias antes do acesso individual às páginas. Este filtro prioriza links do **G1** e verifica a presença de "Santa Catarina" nos links, mas ressalta as inconsistências do padrão de formatação ao longo dos anos.

---

**Funcionamento:**  

1. **Importação e Leitura de Dados:**  
   - Lê o arquivo `Notícias_cidade_regiao_definida.xlsx`, contendo notícias filtradas anteriormente.  

2. **Primeiro Filtro:**  
   - Seleciona apenas notícias com links válidos contendo "https".
   - Retém notícias baseadas em critérios:
     - Publicadas pelo **G1**.
     - Contêm informações de cidades definidas.
     - Incluem "santa-catarina" ou "/sc/" no link.

3. **Eliminação de Outras Regiões:**  
   - Para cada notícia, analisa o texto e link em busca de estados/regiões fora de SC.
   - Notícias com referências a outros estados são descartadas.

4. **Cálculo Estatístico:**  
   - Calcula o número total de notícias e o percentual das filtradas que ainda são relevantes.

5. **Exportação:**  
   - Salva o resultado no arquivo `Filtradas link SC.xlsx` para o próximo passo do processo.

---

**Saída:**  
- **Total de notícias:** Quantidade inicial antes do filtro.  
- **Notícias relevantes:** Quantidade e percentual de notícias filtradas que continuam relevantes.  
- **Exportação:** Arquivo final das notícias aptas para o acesso individual.  

In [ ]:
import pandas as pd
from urllib.parse import unquote

noticias_filtradas = pd.read_excel("./planilhas/Notícias_cidade_regiao_definida.xlsx")

filtered_df = noticias_filtradas[list(map(lambda x: "https" in x , noticias_filtradas["Link"]))]

filtered_df = filtered_df[list(map(lambda x, y, z: z == "G1" or y != "[]" or "santa-catarina" in x.split("https")[1] or "/sc/" in x, filtered_df["Link"], filtered_df["Cidade"], filtered_df["Produto"]))]
print(len(filtered_df))
outraRegiao = [True]*len(filtered_df)

for noticia in range(len(filtered_df)):
    text = str(filtered_df.values[noticia][2]) + str(filtered_df.values[noticia][4])
    link = filtered_df["Link"][noticia]

    for x in RegioesBrasil:
        if x in text:
            outraRegiao[noticia] = False
            break
        if len(x) == 2:
            if "F" + x.lower() + "%" in link:
                outraRegiao[noticia] = False
                print(link)
        elif x.lower() in link or x.lower().replace(" ", "-") in link:
                outraRegiao[noticia] = False

# if filtered_df.values[noticia][]
filtered_df = filtered_df[outraRegiao]

total_noticias = len(noticias_filtradas)

num_noticias_filtradas = len(filtered_df)

percentual_filtradas = (num_noticias_filtradas / total_noticias) * 100

noticias_filtradas = filtered_df
noticias_filtradas.to_excel('./planilhas/Filtradas link SC.xlsx', index=False)

print(f"Total de notícias: {total_noticias}")
print(f"Número de notícias 'G1' ou 'G1 SC': {num_noticias_filtradas}")
print(f"Percentual correspondente: {percentual_filtradas:.2f}%")


### Mini Relatório: Análise do Código

**Objetivo:**  
Este bloco realiza a última etapa do processo de filtragem das notícias. Ele utiliza o acesso direto às páginas de cada notícia para buscar informações mais específicas relacionadas a Santa Catarina (SC). Esse passo é necessário devido à inconsistência de padrões nos dados e links, mas é o mais demorado devido ao tempo necessário para carregar cada página individualmente.

---

**Funcionamento:**  

1. **Configuração do Driver Selenium:**  
   - Inicializa o driver **Selenium** para automação de navegador, utilizando o Chrome em modo headless (sem interface gráfica).  
   - Define configurações para ignorar certificados, notificações e simular um agente de usuário padrão.

2. **Leitura dos Dados:**  
   - Carrega as notícias filtradas no arquivo `Filtradas link SC.xlsx`.  
   - Seleciona apenas as notícias sem cidades previamente definidas, priorizando aquelas que ainda precisam de verificação manual.

3. **Iteração com Acesso às Páginas:**  
   - Utiliza o Selenium para acessar cada link de notícia.  
   - Analisa o conteúdo da página com o **BeautifulSoup**:
     - Busca o título (`h1`) e a editora associados à notícia.
     - Verifica a presença de "Santa Catarina" ou "santa-catarina" no título ou editora.

4. **Marcação das Notícias Relevantes:**  
   - Se a notícia contém referências explícitas a SC no título ou editora, ela é marcada como relevante no vetor `filtro`.

5. **Unificação e Exportação dos Resultados:**  
   - Combina as notícias recém-confirmadas com aquelas já garantidas como relacionadas a SC.  
   - Salva o resultado final no arquivo `Filtragem final.xlsx`.

---

**Saída:**  
- **Notícias confirmadas:** Número de notícias relacionadas a SC encontradas nessa etapa.  
- **Arquivo final:** Contém todas as notícias consideradas relevantes para SC após todas as etapas do processo de filtragem.

**Nota:**  
Essa etapa complementa as anteriores, garantindo maior precisão na classificação, mas tem maior custo computacional devido à necessidade de acessar cada página individualmente.

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from tqdm import tqdm
from urllib.parse import unquote

print("Inicializando driver...")
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-web-security')
options.add_argument('--disable-site-isolation-trials')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument('--disable-notifications')

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

print("Driver iniciado")

verificar_pagina = pd.read_excel("./planilhas/Filtradas link SC.xlsx")
verificar_pagina = verificar_pagina[list(map(lambda x: x == "[]" , verificar_pagina["Cidade"]))]

filtro = [False]*len(verificar_pagina)
cont = 0

with tqdm(total = len(verificar_pagina), desc = "Verificando páginas") as pbar:
    for noticia in verificar_pagina["Link"][::-1][1:60]:

        # print(noticia)
        try:
            driver.get(noticia)
        except:
            print(noticia)
            print("Error timeout")
            cont+=1
            pbar.update(1)
            continue
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        title = soup.find("h1", {"class": "content-head__title"})
        editora = soup.find("a", {"class": "header-editoria--link ellip-line"})
        #
        
        # print(editora)
        # textContent = soup.find_all("p", {"class": "content-text__container"})

        
        
        if not type(title) == str:
            cont+=1
            pbar.update(1)
            continue
            
        if "santa catarina" in title.lower() or "santa catarina" in editora.lower() or "santa-catarina" in editora.lower():
            filtro[cont] = True
        
        cont+=1
        pbar.update(1)

noticias_encontradas = verificar_pagina[filtro]

sc_garantido = pd.read_excel("./planilhas/Filtradas link SC.xlsx")
sc_garantido = sc_garantido[list(map(lambda x: x != "[]" , sc_garantido["Cidade"]))]


pd.concat(noticias_encontradas, sc_garantido)


noticias_encontradas.to_excel("./planilhas/Filtragem final.xlsx", index=False)

print(f"Foram achadas {len(noticias_encontradas)}")